# ATM 115 Computing project
In this class we will learn how to reproduce some of the plots shown in class. While doing so, we will learn how to do some basic analysis of atmospheric simulation data.

## Basic: This is Python, a very popular programming language.
It is:
* Free 
* Open source
* Easy to understand

In [ ]:
1 + 1

In [ ]:
# This is a comment. All text that appears after "#" is ignored.

In [ ]:
#In here I am creating a variable called "a", and with "=" I am assigning the result of 1 + 1 to the variable "a"
a = 1 + 1

In [ ]:
a

## Let us load some useful packages

In [ ]:
import numpy as np
import xarray as xr
import scipy.signal
from matplotlib import pyplot as plt, animation
from IPython.display import HTML, display

In [ ]:
%matplotlib inline

In [ ]:
#I got this dataset from the NOAA website at https://psl.noaa.gov/cgi-bin/db_search/DBSearch.pl?Dataset=NCEP+Reanalysis&Variable=Upward+longwave+radiation+flux&group=0&submit=Search
ds = xr.open_dataset('ulwrf.ntat.gauss.2021.nc')

In [ ]:
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)

In [ ]:
ds

In [ ]:
olr = ds["ulwrf"]

In [ ]:
olr

This dataset has data of outgoing longwave radiation for one year and many latitudes and longitudes. 

# How dows OLR looks like on Earth? What is OLR telling us?

## We can see the value of olr at a given point and time by indexing into the array (think of it as a matrix)

In [ ]:
#Xarray can help you do plots very easily
olr[-1,:,:].plot() #Last time index, all latitudes, all longitudes

# Let us do an average profile to see if we can get a clue

In [ ]:
olr.mean(dim=('time', 'lon')).plot()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
cax = olr[0,:,:].plot()

# Next we need to create a function that updates the values for the colormesh, as well as the title.
def animate(frame):
    cax.set_array(olr[frame,:,:].values.flatten())
    ax.set_title("Time = " + str(olr.coords['time'].values[frame])[:13])

# Finally, we use the animation module to create the animation.
ani = animation.FuncAnimation(
    fig,             # figure
    animate,         # name of the function above
    frames=500,       # Could also be iterable or list
    interval=200     # ms between frames
)

In [ ]:
HTML(ani.to_jshtml())

Let us select only tropical latitudes. For this we use the "sel" method.

In [ ]:
olr_tropics = olr.sel(lat=slice(20,-20))

In [ ]:
olr_anomaly = olr_tropics - olr_tropics.mean(dim=('lon'))

In [ ]:
olr_anomaly[0,:,:].plot()

In [ ]:
hovmoller = olr_anomaly.mean(dim='lat')

In [ ]:
hovmoller.plot()

In [ ]:
# Start wheeler kiladis processing (got help from https://github.com/jamesp/msc_notebooks/blob/master/Convectively%20Coupled%20Waves.ipynb)

In [ ]:
def best_fit(xs, ys):
    """Using the method of least squares, return the gradient
    and y-intercept of the line of best fit through xs and ys."""
    A = np.array([xs, np.ones(len(xs))])
    return np.linalg.lstsq(A.T,ys, rcond = -1)[0]

In [ ]:
fts = []
for j, lat in enumerate(olr_tropics.lat):
    data = olr_tropics[:,j,:]
    lng_avg = data.mean(dim=('lon'))
    m,c = best_fit(range(len(lng_avg)),lng_avg.to_numpy())
    perturbations = data - (m*range(len(lng_avg)) + c)[:, np.newaxis]
    taper = 30
    perturbations[:taper,:] = perturbations[:taper,:] * (np.cos(np.linspace(-np.pi/2, 0, taper))**2)[:, np.newaxis]
    perturbations[-taper:,:] = perturbations[-taper:,:] * (np.cos(np.linspace(0, np.pi/2, taper))**2)[:, np.newaxis]
    lft = np.fft.fft(perturbations, axis=1)     # FFT in space
    tft = np.fft.fft(lft, axis=0)               # FFT in time
    fts.append(np.fft.fftshift(tft))
fts = np.array(fts)
fts = fts[:, :, ::-1]

In [ ]:
def power(spectra):
    return np.abs(spectra)**2


def background(spectra, fsteps=10, ksteps=10):
    """Uses a 1-2-1 filter to generate 'red noise' background field for a spectra (as per WK1998)
        `fsteps` is the number of times to apply the filter in the frequency direction
        `ksteps` is the number of times to apply the filter in the wavenumber direction
    
    Returns a background field of same dimensions as `spectra`.
    """
    # create a 1D 1-2-1 averaging footprint
    bgf = spectra
    for i in range(fsteps):
        # repeated application of the 1-2-1 blur filter to the spectra
        footprint = np.array([[0,1,0], [0,2,0], [0,1,0]]) / 4.0
        bgf = scipy.signal.convolve2d(bgf, footprint, mode='same', boundary='wrap')
    for i in range(ksteps):
        # repeated application of the 1-2-1 blur filter to the spectra
        footprint = np.array([[0,0,0], [1,2,1], [0,0,0]]) / 4.0
        bgf = scipy.signal.convolve2d(bgf, footprint, mode='same', boundary='wrap')
    
    return bgf

def remove_background(spectra):
    """A simple background removal to eliminate frequency noise."""
    bg = background(spectra, fsteps=10, ksteps=10)
    return spectra - bg

In [ ]:
def plot_power_spectra(spectra, dt=None):
    nw, nk = spectra.shape
    T = 6*3600.0*nw/86400.0
    #print(T/86400.) Should be 365
    ks = np.arange(-nk/2, nk/2)  # non-dim wavenumber and frequency
    ws = np.arange(-nw/2, nw/2)
    #rearth =  6371220.00
    #ksd = ks / rearth            # dimmed wavenumber and frequency
    wsd = ws * np.pi*2.0 / T

    plt.pcolormesh(ks, wsd, np.log(spectra),cmap='turbo')

    plt.xlim((-100, 100))    # show up to wavenumber +/- 100 
    plt.ylim((0, 4))          # show positive frequencies, up to one wavelength per day
    plt.xlabel('wavenumber $k$')
    plt.ylabel('frequency $\omega$')

In [ ]:
symmetric_power = np.abs(remove_background(fts.sum(0)))

In [ ]:
plot_power_spectra(symmetric_power)